In [1]:
import dask.dataframe as dd
from dask.dataframe import from_pandas
from dask.dataframe.utils import make_meta
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError
from dask.distributed import Client, LocalCluster, get_worker
import dask

import os
import time
import timeit
from tqdm import tqdm
import pandas as pd
import re
import gc
import numpy as np
import dill
import ctypes
import sys

# Miners
from pm4py import serialize, deserialize
from pm4py import discover_dfg as dfg_discovery
from pm4py.discovery import DFG

from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py import discover_petri_net_inductive as inductive_miner


# Evaluators
from contribution import fitness_alignment, generalization, precision_alignment
from pm4py.algo.evaluation.simplicity import algorithm as simplicity_evaluator #simplicity
from pm4py.objects.petri_net.utils.check_soundness import check_easy_soundness_net_in_fin_marking

In [2]:
dask.config.set({'distributed.scheduler.active-memory-manager.start': True, 'distributed.worker.memory.spill': 0.60})
# dask.config.set(scheduler='processes')

In [3]:
sys.setrecursionlimit(3000)

In [4]:
class graph_driver():
    def __init__(self, uri_scheme='bolt', host='localhost', port='7687', username='neo4j', password='123456'):
        self.uri_scheme = uri_scheme
        self.host = host
        self.port = port
        
        self.username = username
        self.password = password
        
        self.connection_uri = "{uri_scheme}://{host}:{port}".format(uri_scheme=self.uri_scheme, host=self.host, port=self.port)
        self.auth = (self.username, self.password)
        self.driver = GraphDatabase.driver(self.connection_uri, auth=self.auth)
        
    def __del__(self):
        self._close_driver()
    
    def _close_driver(self):
        if self.driver:
            self.driver.close()
    
    def run_single_query(self, query):
        res = None
        with self.driver.session() as session:
            raw_res = session.run(query)
            res = self.format_raw_res(raw_res)
        return res
    
    def run_bulk_query(self, query_list):
        results = []
        with self.driver.session() as session:
            for query in tqdm(query_list):
                raw_res = session.run(query)
                res = self.format_raw_res(raw_res)
                results.append({'query':query, 'result':res})
        return results
    
    def reset_graph(self, db=None):
        return self.run_single_query("MATCH (n) DETACH DELETE n")
    
    def test_connection(self):
        return self.run_single_query("MATCH (n) RETURN COUNT(n) as nodes")
    
    @staticmethod
    def format_raw_res(raw_res):
        res = []
        for r in raw_res:
            res.append(r)
        return res

In [5]:
def useExecutionTime(func):
    
    def compute(*args, **kwargs):
        begin = time.time()
        
        result = func(*args, **kwargs)
        
        end = time.time()
        
        return {"result": result, "execution_time": end - begin}
 
    return compute

@useExecutionTime
def getComputeTime(*args, **kwargs):
    return dask.compute(*args, **kwargs)

In [6]:
cluster = LocalCluster(n_workers=1, threads_per_worker=4, memory_limit=None)

In [7]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37565,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:40059,Total threads: 4
Dashboard: http://127.0.0.1:45823/status,Memory: 0 B
Nanny: tcp://127.0.0.1:37203,


In [8]:
# def trim_memory() -> int:
#     libc = ctypes.CDLL("libc.so.6")
#     return libc.malloc_trim(0)

# def run_gc(dask_worker,**kwargs):
#     gc.collect()
#     return True

# # Register the GC function as a plugin
# client.register_worker_plugin(run_gc, "my_gc_plugin")
# client.register_worker_plugin(trim_memory, "my_trim_plugin")

In [9]:
cluster.adapt(minimum=1, maximum=8)

In [10]:
columnTypes = {
#     'case:IDofConceptCase': 'string',
#     'case:Includes_subCases': 'string',
#     'case:Responsible_actor': 'string',
#     'case:caseProcedure': 'string',
#     'case:concept:name': 'int64',
#     'dueDate': 'object',
#     'case:termName': 'string',
#     'dateStop': 'object',
#     'case:endDate': 'object',
#     'case:endDatePlanned': 'object',
#     'case:parts': 'object',
#     'msgCode': 'string',
#     'msgType': 'string',
#     'case:landRegisterID': 'object'
}

# list of file paths to be loaded

file_paths = ['BPI_2014_Detail Incident Activity']

# load the first file as a Dask dataframe
df = dd.read_csv('{}.csv'.format(file_paths[0]), dtype=columnTypes, encoding="ISO-8859-1", sep=';')

# iterate over the remaining files
for file_path in file_paths[1:]:
    # usecols parameter to load only the columns that are present in both dataframes
    df_temp = dd.read_csv('{}.csv'.format(file_path), dtype=columnTypes)
    # concatenate the dataframes along the rows
    df = dd.concat([df, df_temp], interleave_partitions=True)

# columnTypes = {
#     'OfferID': 'string'
# }


# BPI 2014 sep=''
df = df.rename(columns={"Incident ID": "case:concept:name", "IncidentActivity_Type": "concept:name", "DateStamp": "time:timestamp"})

# df = df.rename(columns={"case concept:name": "case:concept:name", "event concept:name": "concept:name", "event time:timestamp": "time:timestamp"})
for column in df.columns:
    if re.search("[Dd]ate.*|time.*", column):
        df[column] = dask.dataframe.to_datetime(df[column], utc=True)

df['case:concept:name'] = df['case:concept:name'].replace(to_replace="[a-zA-Z]", value='', regex=True)
df['case:concept:name'] = df['case:concept:name'].astype('int')
        
# df = df.repartition(npartitions=1)

In [11]:
def transformToDFG(dfgResult):
    result = {}
    for record in dfgResult:
        result[(record["parent"], record["child"])] = record["frequency"]
    
    return result

def transformToStartEndActivity(activities):
    result = {}
    for record in activities:
        result[record['name']] = record["frequency"]
        
    return result

In [12]:
def getDFG():
    queries = {
        "dfgQuery": """MATCH result=(p:Activity)-[r:PRODUCES]->(c:Activity) RETURN p.name as parent, c.name as child, r.frequency as frequency""",
        "startEndActivitiesQuery": ["MATCH (a:StartActivity) RETURN a.name as name , a.frequency as frequency", "MATCH (a:EndActivity) RETURN a.name as name , a.frequency as frequency"],
    }
    
    neo4jConnection = graph_driver(uri_scheme="neo4j",host="neo4j", password="123456")
    
    dfgResult = neo4jConnection.run_single_query(queries['dfgQuery'])
    startEndActivitiesResult = neo4jConnection.run_bulk_query(queries['startEndActivitiesQuery'])
    return [transformToDFG(dfgResult), transformToStartEndActivity(startEndActivitiesResult[0]["result"]), transformToStartEndActivity(startEndActivitiesResult[1]["result"])]
    

In [13]:
indexed_df = df.set_index('case:concept:name', drop=False, sorted=True)
indexed_df['case:concept:name'] = indexed_df['case:concept:name'].astype({'case:concept:name': 'string'})

In [14]:
del df

In [15]:
indexed_df.index = indexed_df.index.rename('caseId')
indexed_df = indexed_df.repartition(npartitions=6)

In [16]:
dfg, start, end = getDFG()
dfgObj = DFG(dfg, start_activities=start, end_activities=end)

100% 2/2 [00:00<00:00, 329.09it/s]


In [17]:
@useExecutionTime
def getMinerResult(dfg, miner):
    result = {}
    if miner == 'heuristic_miner':
        net, im, fm = heuristics_miner.apply_dfg(dfg['dfg'])
    elif miner == 'inductive_miner':
        net, im, fm = inductive_miner(dfg['dfgObj'])
    elif miner == 'alpha_miner':
        net, im, fm = alpha_miner.apply_dfg(dfg['dfg'])
    
    result[miner] = serialize(net, im, fm)
    
    
    
    return result
    
def setLazyMiners(dfg):
    lazyList = []
    miners = [
        'heuristic_miner',
        'inductive_miner',
        'alpha_miner'
    ]
    for miner in miners:
        task = dask.delayed(getMinerResult)(dfg, miner)
        lazyList.append(task)
    
    return lazyList

def reformatMinersResults(lazyMinersResults):
    minersResults = {}
    for result in lazyMinersResults:
        miner = list(result['result'].keys())[0]
        minersResults[miner] = {}
        net, im, fm = deserialize(result['result'][miner])
        minersResults[miner] = {
            'net': net,
            'im': im,
            'fm': fm,
            'execution_time': result['execution_time']
        }
    return minersResults

In [18]:
@useExecutionTime
def getMetrics(log, miner, metric, net, im, fm):
    sys.setrecursionlimit(3000)
    try:
        result = {}
        result.setdefault(miner, {})
        result[miner].setdefault(metric, 0)
        if metric == 'fitness':
            result[miner][metric] = fitness_alignment.apply(log, net, im, fm)
        elif metric == 'simplicity':
            result[miner][metric] = simplicity_evaluator.apply(net)
        elif metric == 'precision':
            result[miner][metric] = precision_alignment.apply(log, net, im, fm)
        elif metric == 'generalization':
            result[miner][metric] = generalization.apply(log, net, im, fm)

        return result
    except Exception as e:
        return {miner: {metric: {"error": e}}}


def setLazyMetrics(log, miners):
    lazyList = []
    metrics = [
#         'fitness',
#         'simplicity',
#         'precision',
        'generalization'
    ]
    
    for metric in metrics:
        for miner in miners.keys():
            
            net, im, fm = [miners[miner]['net'], miners[miner]['im'], miners[miner]['fm']]
            task = getMetrics(log, miner, metric, net, im, fm)
            lazyList.append(task)
    
    return lazyList

In [19]:
lazyMiners = setLazyMiners({"dfgObj": dfgObj, "dfg": dfg})

In [20]:
lazyMinersResults = reformatMinersResults(dask.compute(*lazyMiners))

In [21]:
@dask.delayed
def aggregate(partitions):
    result = {}
    for partition in partitions:
        for output in partition:
            miner = list(output['result'].keys())[0]
            metric = list(output['result'][miner].keys())[0]
            e_time = output['execution_time']
            
            result.setdefault(miner, {})
            result[miner].setdefault(metric, {})
            result[miner][metric].setdefault('result', None)
            result[miner][metric].setdefault('execution_time', 0)
            
            result[miner][metric]['execution_time'] = e_time
            
            if result[miner][metric]['result'] == None:
                result[miner][metric]['result'] = output['result'][miner][metric]
                continue
    
            if metric and metric == 'fitness':
                result[miner][metric]['result'] = fitness_alignment.aggregate(output['result'][miner][metric], result[miner][metric]['result'])
            elif metric and metric == 'precision':
                result[miner][metric]['result'] = precision_alignment.aggregate(output['result'][miner][metric], result[miner][metric]['result'])
            elif metric and metric == 'generalization':
                result[miner][metric]['result'] = generalization.aggregate([output['result'][miner][metric], result[miner][metric]['result']])

                
    return result

In [22]:
def compute_metrics(aggregatedMetrics, minersResults):
    
    results = {}
    
#     getMinerResultByMiner = lambda results, miner: [value for value in lazyMinersResults if list(value['result'].keys())[0] == miner].pop()
    
    for miner, metrics in aggregatedMetrics.items():
#         net, im, fm = deserialize(getMinerResultByMiner(minersResults, miner)['result'][miner])
        net, im, fm = [minersResults[miner]['net'], minersResults[miner]['im'], minersResults[miner]['fm']]
        for metricKey, metricValue in metrics.items():
            results.setdefault(miner, {})
            results[miner].setdefault(metricKey, {})
            results[miner][metricKey].setdefault('result', None)
            results[miner][metricKey].setdefault('execution_time', 0)
            
            results[miner][metricKey]['execution_time'] = metricValue['execution_time']
            start_time = timeit.default_timer()
            if not check_easy_soundness_net_in_fin_marking(net, im, fm) and metricKey == 'precision':
                results[miner][metricKey]['result'] = 'NA'
                continue
            if metricKey and metricKey == 'fitness':
                results[miner][metricKey]['result'] = fitness_alignment.compute(metricValue['result'], net=net, im=im, fm=fm)
            elif metricKey and metricKey == 'precision':
                results[miner][metricKey]['result'] = precision_alignment.compute(**metricValue['result'], net=net, im=im, fm=fm)
            elif metricKey and metricKey == 'generalization':
                results[miner][metricKey]['result'] = generalization.compute(metricValue['result']['aligned_traces'], net=net)
            elif metricKey and metricKey == 'simplicity':
                results[miner][metricKey]['result'] = simplicity_evaluator.apply(net)
            end_time = timeit.default_timer()
            results[miner][metricKey]['execution_time'] = sum([(end_time - start_time), metricValue['execution_time']])
    return results

In [23]:
mapped_data = indexed_df.map_partitions(setLazyMetrics, lazyMinersResults, meta=[])

In [24]:
aggregated_results = aggregate(mapped_data)

In [25]:
r = dask.delayed(compute_metrics)(aggregated_results, lazyMinersResults)

In [26]:
results = dask.compute(r)

replaying log with TBR, completed variants ::   1%|          | 72/6006 [00:00<00:30, 191.88it/s]

replaying log with TBR, completed variants ::   2%|▏         | 104/6035 [00:00<00:26, 227.94it/s]

replaying log with TBR, completed variants ::   2%|▏         | 128/6035 [00:00<00:32, 180.66it/s]

replaying log with TBR, completed variants ::   2%|▏         | 108/6006 [00:00<00:48, 120.90it/s][A


replaying log with TBR, completed variants ::   2%|▏         | 148/6035 [00:00<00:43, 136.45it/s]

replaying log with TBR, completed variants ::   1%|          | 65/6236 [00:00<00:45, 134.96it/s]


replaying log with TBR, completed variants ::   0%|          | 18/6149 [00:00<00:42, 144.82it/s]

replaying log with TBR, completed variants ::   1%|▏         | 87/6236 [00:00<00:38, 159.10it/s]


replaying log with TBR, completed variants ::   3%|▎         | 164/6035 [00:01<00:48, 120.15it/s]


replaying log with TBR, completed variants ::   3%|▎         | 178/6035 [00:01<00:47, 123.31it/s]

replayin

replaying log with TBR, completed variants ::  22%|██▏       | 1221/5541 [00:04<00:17, 246.02it/s]A

replaying log with TBR, completed variants ::  23%|██▎       | 1247/5541 [00:05<00:17, 240.22it/s]A


replaying log with TBR, completed variants ::   9%|▊         | 527/6149 [00:08<04:16, 21.88it/s]

replaying log with TBR, completed variants ::  11%|█▏        | 681/6006 [00:09<01:49, 48.51it/s]s]A


replaying log with TBR, completed variants ::   9%|▊         | 536/6149 [00:08<02:50, 32.93it/s]

replaying log with TBR, completed variants ::  12%|█▏        | 695/6006 [00:09<01:19, 66.58it/s]s]A


replaying log with TBR, completed variants ::  12%|█▏        | 708/6006 [00:09<01:13, 72.31it/s]


replaying log with TBR, completed variants ::  25%|██▍       | 1360/5541 [00:05<00:17, 244.74it/s]A

replaying log with TBR, completed variants ::  12%|█▏        | 719/6006 [00:09<01:15, 69.69it/s]


replaying log with TBR, completed variants ::  25%|██▍       | 1385/5541 [00:05<00:17, 231.96it/s]

replaying log with TBR, completed variants ::  36%|███▌      | 1978/5541 [00:09<00:21, 167.36it/s]A


replaying log with TBR, completed variants ::  15%|█▍        | 894/6006 [00:13<02:22, 36.00it/s]

replaying log with TBR, completed variants ::  36%|███▋      | 2016/5541 [00:09<00:21, 165.89it/s]A


replaying log with TBR, completed variants ::  15%|█▌        | 909/6006 [00:13<01:41, 50.11it/s]


replaying log with TBR, completed variants ::  37%|███▋      | 2076/5541 [00:09<00:20, 173.03it/s]A


replaying log with TBR, completed variants ::  15%|█▌        | 928/6006 [00:14<01:33, 54.28it/s]

replaying log with TBR, completed variants ::  38%|███▊      | 2096/5541 [00:10<00:19, 176.48it/s]A


replaying log with TBR, completed variants ::  12%|█▏        | 726/6149 [00:13<02:18, 39.22it/s]

replaying log with TBR, completed variants ::  14%|█▎        | 825/6035 [00:14<01:44, 49.96it/s]


replaying log with TBR, completed variants ::  12%|█▏        | 731/6149 [00:13<02:17, 39.30it/s]

re

replaying log with TBR, completed variants ::  19%|█▊        | 1123/6006 [00:18<01:45, 46.41it/s]]A


replaying log with TBR, completed variants ::  15%|█▌        | 927/6035 [00:18<03:23, 25.05it/s]

replaying log with TBR, completed variants ::  19%|█▉        | 1152/6006 [00:18<00:58, 82.54it/s]]A


replaying log with TBR, completed variants ::  19%|█▉        | 1162/6006 [00:18<00:57, 84.64it/s]]A

replaying log with TBR, completed variants ::  16%|█▌        | 990/6236 [00:18<01:43, 50.93it/s]


replaying log with TBR, completed variants ::  20%|█▉        | 1175/6006 [00:19<00:50, 95.68it/s]]A

replaying log with TBR, completed variants ::  16%|█▌        | 999/6236 [00:18<01:29, 58.75it/s]


replaying log with TBR, completed variants ::  59%|█████▊    | 3252/5541 [00:14<00:05, 397.21it/s]A

replaying log with TBR, completed variants ::  15%|█▌        | 931/6035 [00:19<04:42, 18.09it/s]


replaying log with TBR, completed variants ::  20%|█▉        | 1186/6006 [00:19<01:10, 68.47it/s]]

replaying log with TBR, completed variants ::  21%|██▏       | 1294/6035 [00:27<02:40, 29.47it/s]

replaying log with TBR, completed variants ::  23%|██▎       | 1454/6236 [00:27<01:31, 52.44it/s]


replaying log with TBR, completed variants ::  91%|█████████ | 5056/5541 [00:23<00:01, 263.12it/s][A

replaying log with TBR, completed variants ::  22%|██▏       | 1309/6035 [00:27<01:58, 39.86it/s]


replaying log with TBR, completed variants ::  92%|█████████▏| 5096/5541 [00:23<00:01, 298.56it/s][A

replaying log with TBR, completed variants ::  22%|██▏       | 1314/6035 [00:27<01:55, 40.86it/s]


replaying log with TBR, completed variants ::  93%|█████████▎| 5128/5541 [00:23<00:01, 284.54it/s][A

replaying log with TBR, completed variants ::  27%|██▋       | 1613/6006 [00:28<03:40, 19.97it/s]

replaying log with TBR, completed variants ::  94%|█████████▍| 5210/5541 [00:24<00:01, 236.83it/s][A

replaying log with TBR, completed variants ::  24%|██▍       | 1490/6236 [00:27<01:55, 41.07it

...

replaying log with TBR, completed variants :: 100%|█████████▉| 5983/6006 [00:13<00:00, 318.42it/s]

replaying log with TBR, completed variants :: 100%|██████████| 6006/6006 [00:13<00:00, 444.56it/s]



replaying log with TBR, completed variants ::  82%|████████▏ | 4945/6035 [00:11<00:03, 329.49it/s]

replaying log with TBR, completed variants ::  43%|████▎     | 2687/6236 [00:05<00:08, 395.86it/s]


replaying log with TBR, completed variants ::  83%|████████▎ | 5028/6035 [00:11<00:02, 411.75it/s]

replaying log with TBR, completed variants ::  85%|████████▍ | 5119/6035 [00:11<00:01, 496.34it/s]

replaying log with TBR, completed variants ::  86%|████████▌ | 5196/6035 [00:11<00:01, 554.91it/s]

replaying log with TBR, completed variants ::  47%|████▋     | 2933/6236 [00:06<00:05, 631.60it/s]

replaying log with TBR, completed variants ::  48%|████▊     | 3017/6236 [00:06<00:04, 678.60it/s]


replaying log with TBR, completed variants ::  87%|████████▋ | 5263/6035 [00:11<00:01, 513.26it/

replaying log with TBR, completed variants ::  78%|███████▊  | 4858/6236 [00:09<00:02, 594.59it/s]


replaying log with TBR, completed variants ::   8%|▊         | 422/5379 [00:03<00:48, 102.61it/s]

replaying log with TBR, completed variants ::   8%|▊         | 436/5379 [00:03<00:46, 106.70it/s]]

replaying log with TBR, completed variants ::   8%|▊         | 449/5379 [00:03<00:44, 110.63it/s]


replaying log with TBR, completed variants :: 100%|█████████▉| 6137/6149 [02:22<00:00, 22.93it/s]

replaying log with TBR, completed variants ::  13%|█▎        | 790/6035 [00:01<00:11, 468.98it/s]


replaying log with TBR, completed variants :: 100%|█████████▉| 6140/6149 [02:22<00:00, 23.40it/s]

replaying log with TBR, completed variants ::  82%|████████▏ | 5102/6236 [00:10<00:02, 553.08it/s]


replaying log with TBR, completed variants ::  14%|█▍        | 843/6035 [00:01<00:11, 438.97it/s]

replaying log with TBR, completed variants ::   9%|▉         | 474/5379 [00:03<00:50, 96.98it/s] ]


r

replaying log with TBR, completed variants ::  12%|█▏        | 645/5379 [00:07<01:04, 73.81it/s]s]


replaying log with TBR, completed variants ::  12%|█▏        | 655/5379 [00:07<00:58, 80.14it/s]s]


replaying log with TBR, completed variants ::  43%|████▎     | 2611/6035 [00:05<00:06, 553.42it/s]


replaying log with TBR, completed variants ::  12%|█▏        | 664/5379 [00:07<01:23, 56.38it/s]s]


replaying log with TBR, completed variants ::  13%|█▎        | 674/5379 [00:07<01:12, 65.14it/s]s]


replaying log with TBR, completed variants ::  13%|█▎        | 684/5379 [00:07<01:07, 69.81it/s]s]


replaying log with TBR, completed variants ::  51%|█████     | 3083/6035 [00:05<00:04, 677.46it/s]


replaying log with TBR, completed variants ::  53%|█████▎    | 3189/6035 [00:06<00:03, 773.92it/s]


replaying log with TBR, completed variants ::  54%|█████▍    | 3282/6035 [00:06<00:03, 785.69it/s]


replaying log with TBR, completed variants ::  56%|█████▌    | 3376/6035 [00:06<00:03, 826.

replaying log with TBR, completed variants ::  99%|█████████▉| 5962/6035 [00:10<00:00, 710.57it/s]


replaying log with TBR, completed variants :: 100%|██████████| 6035/6035 [00:10<00:00, 585.67it/s]



replaying log with TBR, completed variants ::  18%|█▊        | 987/5379 [00:12<00:45, 96.18it/s]s]


replaying log with TBR, completed variants ::  19%|█▊        | 999/5379 [00:12<00:44, 98.52it/s]s]


replaying log with TBR, completed variants ::  19%|█▉        | 1010/5379 [00:12<00:46, 93.53it/s]]


replaying log with TBR, completed variants ::  19%|█▉        | 1024/5379 [00:12<00:42, 102.51it/s]


replaying log with TBR, completed variants ::  19%|█▉        | 1047/5379 [00:12<00:33, 129.59it/s]


replaying log with TBR, completed variants ::  20%|█▉        | 1061/5379 [00:13<00:34, 125.83it/s]


replaying log with TBR, completed variants ::  20%|█▉        | 1074/5379 [00:13<00:34, 125.71it/s]


replaying log with TBR, completed variants ::  20%|██        | 1092/5379 [00:13<00:31, 138

replaying log with TBR, completed variants :: 100%|██████████| 5379/5379 [00:02<00:00, 2122.69it/s]


In [27]:
results

({'heuristic_miner': {'generalization': {'result': -0.34603392506570896,
    'execution_time': 36.94729153322987}},
  'inductive_miner': {'generalization': {'result': -0.18967788656340723,
    'execution_time': 3.2223896235227585}},
  'alpha_miner': {'generalization': {'result': -0.20328507213233138,
    'execution_time': 2.7243997189216316}}},)

2023-05-12 01:00:12,019 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:39913'.
2023-05-12 01:00:12,022 - distributed.worker - ERROR - Scheduler was unaware of this worker 'tcp://127.0.0.1:39913'. Shutting down.
2023-05-12 01:00:12,035 - tornado.application - ERROR - Exception in callback functools.partial(<function TCPServer._handle_connection.<locals>.<lambda> at 0xffff736a3700>, <Task finished name='Task-682' coro=<BaseTCPListener._handle_stream() done, defined at /opt/conda/lib/python3.9/site-packages/distributed/comm/tcp.py:553> exception=ValueError('invalid operation on non-started TCPListener')>)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/opt/conda/lib/python3.9/site-packages/tornado/tcpserver.py", line 331, in <lambda>
    gen.convert_yielded(future), lambda f: f.result()
  File "/opt/conda/lib/python3.9/site-packag